In [8]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords 
nltk.download('stopwords')
import string
string.punctuation
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from copy import deepcopy as dp
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords  
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.externals import joblib
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from copy import deepcopy

[nltk_data] Downloading package punkt to
[nltk_data]     /home/parasmehan123/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/parasmehan123/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/parasmehan123/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df1=pd.read_csv('data.csv')

In [3]:
df1.head()

,title,score,id,url,comms_num,created,body,flair
0,Rahul Gandhi resigns as President of Congress,1120,c8lydy,https://www.aninews.in/news/national/general-n...,413,1.562175e+09,NaN,0
1,Is legalization of marijuana in India ever goi...,81,bvvre5,https://www.reddit.com/r/india/comments/bvvre5...,124,1.559497e+09,"As a cannabis enthusiast (stoner, pothead), it...",0
2,I was searching for something and I stumbled u...,314,cd3xmx,https://i.imgur.com/FefhpHc.jpg,109,1.563145e+09,NaN,0
3,Emperors of Extraction: The Mughals did not ma...,35,cdargd,https://www.dailyo.in/politics/emperors-of-ext...,7,1.563181e+09,NaN,0
4,Chennaite here. Is there any way for us to mak...,83,c22c7k,https://www.reddit.com/r/india/comments/c22c7k...,30,1.560894e+09,It's crazy how they aren't even talking about ...,0


In [4]:
title=df1['title'].tolist()
body=df1['body'].tolist()
url=df1['url'].tolist()
y=df1['flair'].tolist()

In [9]:

def stopwords_removal(tokens):
    stop_words = list(stopwords.words("english"))
    filtered_sentence = [w for w in tokens if not w in stop_words]              
    final=' '.join(filtered_sentence)
    return final

def remove_punctuation(text):
    obj=str.maketrans('', '',string.punctuation)
    answer=text.translate(obj)
    return answer

def simple_lemma(data):
    lemmatizer = WordNetLemmatizer()
    filtered=[lemmatizer.lemmatize(word) for word in data.split()]
    text = ' '.join(filtered)
    return text

def pre_process(y,flag):
    x=deepcopy(y)
    for i in range(len(x)):
        if str([i])!='nan':
            if flag:
                x[i]=x[i].split('/')
                x[i]=' '.join(x[i])
            x[i]=simple_lemma(remove_punctuation(stopwords_removal(word_tokenize(str(x[i]).lower()))))
        else:
            x[i]=''
    return x

In [10]:
title[:5]

['Rahul Gandhi resigns as President of Congress',
 'Is legalization of marijuana in India ever going to be a serious political issue during our lifetime, or are we going to be criminals forever?',
 'I was searching for something and I stumbled upon my 2nd std papers! 9¾/10 bois, AMA 😂',
 'Emperors of Extraction: The Mughals did not make India rich. Claims of their welfarism only buttress a political agenda',
 'Chennaite here. Is there any way for us to make the political parties talk about building ponds and reservoirs?']

In [11]:
title_clean=pre_process(title,False)
body_clean=pre_process(body,False)
url_clean=pre_process(url,True)

In [12]:
body_clean[:5]

['nan',
 'cannabis enthusiast stoner pothead pain see many country across world taking issue seriously re reduced criminal status purchasing illegal weed questionable source expect india take debate anytime soon',
 'nan',
 'nan',
 's crazy nt even talking even opposition ca nt go like p situation grim chennai somehow sound worse medium guess could say dire situation nt case chennai depends locality ground water run almost run locality every year situation gradually worsening guess']

In [13]:
def train(classifier,X_train,y_train,X_test,y_test):
    classifier.fit(X_train,y_train)
    y_train_pred=classifier.predict(X_train)
    y_test_pred=classifier.predict(X_test)
    return accuracy_score(y_train,y_train_pred),accuracy_score(y_test,y_test_pred)

In [14]:
mst=(np.random.rand(len(df1)) < 0.8)

In [15]:

def train_test(x):
    X_train=[]
    X_test=[]
    y1_train=[]
    y1_test=[]
    for i in range(len(mst)):
        if mst[i]:
            X_train.append(x[i])
            y1_train.append(y[i])
        else:
            X_test.append(x[i])
            y1_test.append(y[i])
    tf = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
    X1_train = tf.fit_transform(X_train)  
    X1_test = tf.transform(X_test)
    
    clf1=GridSearchCV(SVC(gamma='scale'),{'decision_function_shape':['ovo','ovr'],},cv=5,iid=False)
    print(train(clf1,X1_train,y1_train,X1_test,y1_test))
    
    clf2=GridSearchCV(KNeighborsClassifier(),{'n_neighbors':[2,5,7]},cv=5,iid=False)
    print(train(clf2,X1_train,y1_train,X1_test,y1_test))
    
    clf3=GridSearchCV(DecisionTreeClassifier(random_state=0),{'criterion':['gini','entropy']},cv=5,iid=False)
    print(train(clf3,X1_train,y1_train,X1_test,y1_test))
    
    clf4=GridSearchCV(LogisticRegression(random_state=0),{'solver':['newton-cg', 'lbfgs','sag', 'saga','lbfgs'],'multi_class':['multinomial'],'max_iter':[200,250,300]},cv=5,iid=False)
    print(train(clf4,X1_train,y1_train,X1_test,y1_test))
    
    clf5=GridSearchCV(RandomForestClassifier(random_state=0),{'n_estimators':[10,50,100,150],'criterion':['gini','entropy']},cv=5,iid=False)
    print(train(clf5,X1_train,y1_train,X1_test,y1_test))
    
    clf6=GridSearchCV(xgb.XGBClassifier(objective="binary:logistic", random_state=42),{'max_depth':[2,3],'learning_rate':[0.1,0.15,0.2]},cv=5,iid=False)
    print(train(clf6,X1_train,y1_train,X1_test,y1_test))
    

In [17]:
train_test(title_clean)

(0.8760217983651226, 0.6467236467236467)
(0.5429155313351499, 0.34472934472934474)
(0.920299727520436, 0.5641025641025641)
(0.7922343324250681, 0.6410256410256411)
(0.920299727520436, 0.6125356125356125)
(0.7316076294277929, 0.5783475783475783)


In [18]:
train_test(url_clean)

(0.26634877384196187, 0.27635327635327633)
(0.21934604904632152, 0.19658119658119658)
(0.26634877384196187, 0.2706552706552707)
(0.2622615803814714, 0.2792022792022792)
(0.26634877384196187, 0.2678062678062678)
(0.26430517711171664, 0.2564102564102564)


In [19]:
train_test(body_clean)

(0.7568119891008175, 0.4985754985754986)
(0.27520435967302453, 0.21937321937321938)
(0.7752043596730245, 0.5327635327635327)
(0.6559945504087193, 0.5242165242165242)
(0.7752043596730245, 0.5783475783475783)
(0.7275204359673024, 0.603988603988604)


In [20]:
ti_bo=[]
for i in range(len(title_clean)):
    ti_bo.append(title_clean[i]+" "+body_clean[i])
train_test(ti_bo)

(0.9407356948228883, 0.717948717948718)
(0.6907356948228883, 0.5441595441595442)
(0.9550408719346049, 0.7549857549857549)
(0.8807901907356949, 0.7435897435897436)
(0.9550408719346049, 0.7863247863247863)
(0.9250681198910081, 0.8176638176638177)


In [21]:
ti_bo_ur=[]
for i in range(len(title_clean)):
    ti_bo_ur.append(title_clean[i]+" "+body_clean[i]+" "+url_clean[i])
train_test(ti_bo_ur)

(0.9455040871934605, 0.7264957264957265)
(0.694141689373297, 0.5498575498575499)
(0.9584468664850136, 0.7663817663817664)
(0.8930517711171662, 0.7549857549857549)
(0.9584468664850136, 0.7948717948717948)
(0.9066757493188011, 0.811965811965812)


In [22]:
bo_ur=[]
for i in range(len(body_clean)):
    bo_ur.append(body_clean[i]+" "+url_clean[i])
train_test(bo_ur)


(0.8072207084468664, 0.5441595441595442)
(0.4019073569482289, 0.28774928774928776)
(0.8228882833787466, 0.5584045584045584)
(0.7173024523160763, 0.5641025641025641)
(0.8228882833787466, 0.6068376068376068)
(0.7084468664850136, 0.6125356125356125)


In [23]:
ti_ur=[]
for i in range(len(title_clean)):
    ti_ur.append(title_clean[i]+" "+url_clean[i])
train_test(ti_ur)


(0.8876021798365122, 0.6552706552706553)
(0.5381471389645777, 0.3418803418803419)
(0.9291553133514986, 0.5754985754985755)
(0.8072207084468664, 0.6609686609686609)
(0.9291553133514986, 0.6410256410256411)
(0.776566757493188, 0.5954415954415955)


In [52]:
clf=GridSearchCV(xgb.XGBClassifier(objective="binary:logistic", random_state=42),{'max_depth':[2,3],'learning_rate':[0.1,0.15,0.2]},cv=5,iid=False)
    

In [53]:
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(ti_bo_ur).toarray()  


In [54]:
clf.fit(X,y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'max_depth': [2, 3], 'learning_rate': [0.1, 0.15, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [55]:
y_pred=clf.predict(X)
print(accuracy_score(y,y_pred))

0.9263331500824629


In [56]:
joblib.dump(clf, 'model.joblib')

['model.joblib']

In [57]:
pd.DataFrame(ti_bo_ur,columns=['text']).to_csv('data_cleaned.csv',index=False)